In [2]:
import torch

in_channeles, out_channels = 5, 10
width, height = 100, 100
kernel_size = 3
batch_size = 1

input = torch.randn(batch_size, in_channeles, width, height)

conv_layer = torch.nn.Conv2d(in_channeles, out_channels, kernel_size)
output = conv_layer(input)

print(input.shape)
print(output.shape)
print(conv_layer.weight.shape)
# torch.Size([10, 5, 3, 3])
#     输出通道数，输入通道数，卷积核大小

torch.Size([1, 5, 100, 100])
torch.Size([1, 10, 98, 98])
torch.Size([10, 5, 3, 3])


In [7]:
# padding = 1

import torch

input =[3,4,6,5,7,
        2,4,6,8,2,
        1,6,7,8,4,
        9,7,4,6,2,
        3,7,5,4,1]

input = torch.Tensor(input).view(1,1,5,5)

conv_layer = torch.nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, padding=1, bias=False)

kernel = torch.Tensor([1,2,3,4,5,6,7,8,9]).view(1,1,3,3)
conv_layer.weight.data = kernel.data

output = conv_layer(input)

print(output)

tensor([[[[ 91., 168., 224., 215., 127.],
          [114., 211., 295., 262., 149.],
          [192., 259., 282., 214., 122.],
          [194., 251., 253., 169.,  86.],
          [ 96., 112., 110.,  68.,  31.]]]], grad_fn=<ConvolutionBackward0>)


In [8]:
# stride = 2

import torch

input =[3,4,6,5,7,
        2,4,6,8,2,
        1,6,7,8,4,
        9,7,4,6,2,
        3,7,5,4,1]

input = torch.Tensor(input).view(1,1,5,5)

conv_layer = torch.nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, stride=2, bias=False)

kernel = torch.Tensor([1,2,3,4,5,6,7,8,9]).view(1,1,3,3)
conv_layer.weight.data = kernel.data

output = conv_layer(input)

print(output)

tensor([[[[211., 262.],
          [251., 169.]]]], grad_fn=<ConvolutionBackward0>)


In [9]:
# max pooling layer

import torch

input = [3,4,6,5,
        2,4,6,8,
        1,6,7,8,
        9,7,4,6,]

input = torch.tensor(input).view(1,1,4,4)

# max pooling layer
maxpooling_layer = torch.nn.MaxPool2d(kernel_size=2)

output = maxpooling_layer(input)

print(output)

tensor([[[[4, 8],
          [9, 8]]]])


In [19]:
# 一个简单的卷积神经网络
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.pooling = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.pooling(F.relu(self.conv1(x)))
        x = self.pooling(F.relu(self.conv2(x)))
        x = x.view(batch_size, -1)
        x = self.fc(x)
        return x

batch_size = 64
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

train_dataset = datasets.MNIST(root='./datasets/mnist', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='./datasets/mnist', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model = Net()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

def train(epoch):
    model.train()
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        inputs, target = data
        inputs, target = inputs.to(device), target.to(device)
        optimizer.zero_grad()

        # forward + backward + update
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, batch_idx + 1, running_loss / 300))
            running_loss = 0.0

def test():
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

if __name__ == '__main__':
    for epoch in range(10):
        train(epoch)
        test()

d:\ProgramData\miniconda3\envs\torch\lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


[1,   300] loss: 0.635
[1,   600] loss: 0.192
[1,   900] loss: 0.153
Accuracy of the network on the 10000 test images: 96 %
[2,   300] loss: 0.119
[2,   600] loss: 0.099
[2,   900] loss: 0.090
Accuracy of the network on the 10000 test images: 97 %
[3,   300] loss: 0.080
[3,   600] loss: 0.076
[3,   900] loss: 0.075
Accuracy of the network on the 10000 test images: 98 %
[4,   300] loss: 0.068
[4,   600] loss: 0.064
[4,   900] loss: 0.061
Accuracy of the network on the 10000 test images: 98 %
[5,   300] loss: 0.058
[5,   600] loss: 0.052
[5,   900] loss: 0.058
Accuracy of the network on the 10000 test images: 98 %
[6,   300] loss: 0.052
[6,   600] loss: 0.052
[6,   900] loss: 0.052
Accuracy of the network on the 10000 test images: 98 %
[7,   300] loss: 0.043
[7,   600] loss: 0.048
[7,   900] loss: 0.049
Accuracy of the network on the 10000 test images: 98 %
[8,   300] loss: 0.045
[8,   600] loss: 0.042
[8,   900] loss: 0.043
Accuracy of the network on the 10000 test images: 98 %
[9,   30